In [ ]:
import time
import os
import requests
import json
from bs4 import BeautifulSoup
from zenrows import ZenRowsClient
from tqdm import tqdm

In [ ]:
# import sitemap  from SAQ
# https://www.saq.com/media/sitemaps/en/sitemap_product_001.xml
# https://www.saq.com/media/sitemaps/en/sitemap_product_002.xml
# https://www.saq.com/media/sitemaps/fr/sitemap_product_002.xml
# https://www.saq.com/media/sitemaps/fr/sitemap_product_001.xml


sitemap_urls = [
    "https://www.saq.com/media/sitemaps/en/sitemap_product_001.xml",
    "https://www.saq.com/media/sitemaps/en/sitemap_product_002.xml",
    "https://www.saq.com/media/sitemaps/fr/sitemap_product_001.xml",
    "https://www.saq.com/media/sitemaps/fr/sitemap_product_002.xml",
]



i=1 
for  url in sitemap_urls :
    # Get the sitemap content
    response = requests.get(url)
    sitemap_content = response.content

    #save sitmap content to file
    with open(f"../data/saq/sitemap_{i}.xml", "wb") as f:
        f.write(sitemap_content)
    i = i + 1

   



In [8]:
from bs4 import BeautifulSoup

# Load the sitemap file (adjust path if needed)
with open("../data/saq/sitemap/sitemap_fr_2.xml", "r", encoding="utf-8") as f:
    sitemap_content = f.read()

# Parse the XML
soup = BeautifulSoup(sitemap_content, "xml")

# Extract all <loc> tags (these contain the product URLs)
urls = [loc.text for loc in soup.find_all("loc")]

# Show first few URLs to explore
print(f"Found {len(urls)} URLs in the sitemap.")
for url in urls[:100]:  # just preview the first 10
    print(url)


Found 25026 URLs in the sitemap.
https://www.saq.com/fr/14947078
https://www.saq.com/media/catalog/product/1/4/14947078-1_1652904633.png?optimize=high&fit=bounds&height=&width=&canvas=:&format=jpeg
https://www.saq.com/fr/14947094
https://www.saq.com/media/catalog/product/1/4/14947094-1_1643994661.png?optimize=high&fit=bounds&height=&width=&canvas=:&format=jpeg
https://www.saq.com/fr/14947481
https://www.saq.com/media/catalog/product/1/4/14947481-1_1646927157.png?optimize=high&fit=bounds&height=&width=&canvas=:&format=jpeg
https://www.saq.com/fr/14947490
https://www.saq.com/media/catalog/product/1/4/14947490-1_1646927157.png?optimize=high&fit=bounds&height=&width=&canvas=:&format=jpeg
https://www.saq.com/fr/14788222
https://www.saq.com/media/catalog/product/1/4/14788222-1_1678907583.png?optimize=high&fit=bounds&height=&width=&canvas=:&format=jpeg
https://www.saq.com/fr/14951085
https://www.saq.com/media/catalog/product/1/4/14951085-1_1689013088.png?optimize=high&fit=bounds&height=&width

In [9]:
filtered_urls =  [url for url in urls if not url.startswith("https://www.saq.com/media/")]
# Show first few URLs to explore
print(f"Found {len(filtered_urls)} URLs in the sitemap.")
for url in filtered_urls[:100]:  # just preview the first 10
    print(url)

Found 12005 URLs in the sitemap.
https://www.saq.com/fr/14947078
https://www.saq.com/fr/14947094
https://www.saq.com/fr/14947481
https://www.saq.com/fr/14947490
https://www.saq.com/fr/14788222
https://www.saq.com/fr/14951085
https://www.saq.com/fr/14953734
https://www.saq.com/fr/14938163
https://www.saq.com/fr/14938171
https://www.saq.com/fr/14954059
https://www.saq.com/fr/14955182
https://www.saq.com/fr/14951886
https://www.saq.com/fr/14951878
https://www.saq.com/fr/14951907
https://www.saq.com/fr/14951894
https://www.saq.com/fr/14956020
https://www.saq.com/fr/14956046
https://www.saq.com/fr/14956038
https://www.saq.com/fr/14956062
https://www.saq.com/fr/14956054
https://www.saq.com/fr/14956071
https://www.saq.com/fr/14956089
https://www.saq.com/fr/14956100
https://www.saq.com/fr/14956097
https://www.saq.com/fr/14956134
https://www.saq.com/fr/14956118
https://www.saq.com/fr/14956151
https://www.saq.com/fr/14956126
https://www.saq.com/fr/14956142
https://www.saq.com/fr/14956193
https:/

In [14]:
import requests
from bs4 import BeautifulSoup
import re
import json

def extract_breadcrumb(script_text):
    match = re.search(r'"ecomm_category":\s*"([^"]+)"', script_text)
    return match.group(1) if match else None

def extract_product_details(soup):
    details = {}
    for li in soup.select("ul.list-attributs li"):
        key = li.find("span").get_text(strip=True)
        val = li.find("strong").get_text(strip=True)
        details[key] = val
    return details

def extract_tasting_notes(soup):
    notes = {}
    tasting_section = soup.select_one("#tasting")
    if tasting_section:
        for li in tasting_section.select("ul.tasting-container li"):
            label = li.find("span", class_="in-line") or li.find("span")
            value = li.find("strong")
            if label and value:
                notes[label.get_text(strip=True)] = value.get_text(strip=True)
    return notes

def extract_pairings(soup):
    pairings = []
    for item in soup.select("#product-data-item-product-recipe li.item.product"):
        name = item.select_one(".product-item-name span")
        url = item.select_one("a.product-item-link")["href"] if item.select_one("a.product-item-link") else None
        pairings.append({"name": name.text.strip(), "url": url})
    return pairings

def extract_product_name(soup):
    h1 = soup.select_one("h1.page-title")
    return h1.get_text(strip=True) if h1 else None


def scrape_saq_product(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    # Extract components
    script_tags = soup.find_all("script")
    product_name = extract_product_name(soup)
    breadcrumb = next((extract_breadcrumb(s.text) for s in script_tags if "ecomm_category" in s.text), None)
    product_details = extract_product_details(soup)
    tasting_notes = extract_tasting_notes(soup)
    pairings = extract_pairings(soup)

    return {
        "url": url,
        "product_name": product_name,
        "breadcrumb": breadcrumb,
        "product_details": product_details,
        "tasting_notes": tasting_notes,
        "pairings": pairings
    }




In [15]:

for url in filtered_urls[:10]:  # just preview the first 10
    print(f"Scraping {url}...")
    product_data = scrape_saq_product(url)
    print(json.dumps(product_data, indent=2, ensure_ascii=False))
    time.sleep(2)  # be polite and don't hammer the server


Scraping https://www.saq.com/fr/14947078...
{
  "url": "https://www.saq.com/fr/14947078",
  "product_name": "Domaine Merlin Saint-Véran Le Grand Bussière",
  "breadcrumb": "Produits > Vin > Vin blanc",
  "product_details": {
    "Pays": "France",
    "Région": "Bourgogne, Mâconnais",
    "Appellation d'origine": "Saint-Véran",
    "Désignation réglementée": "(AOC/AOP) Appellation origine controlée/protégée",
    "Cépage": "Chardonnay 100 %",
    "Degré d'alcool": "13,5 %",
    "Taux de sucre": "1,2 g/L",
    "Couleur": "Blanc",
    "Format": "750 ml",
    "Producteur": "Merlin SARL",
    "Agent promotionnel": "Vins Balthazard Inc.",
    "Code SAQ": "14947078",
    "Code CUP": "03760201540916"
  },
  "tasting_notes": {},
  "pairings": []
}
Scraping https://www.saq.com/fr/14947094...
{
  "url": "https://www.saq.com/fr/14947094",
  "product_name": "La Porte Saint-Jean Le Saut Mignon 2020",
  "breadcrumb": "Produits > Vin > Vin blanc",
  "product_details": {
    "Pays": "France",
    "Dési